In [1]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime

# Inizializzare MetaTrader 5
login = 90026240
password = '@fElE5Vr'
server = 'easyMarkets-Live'
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_M5 
strategy_name = 'vwap_engulfing'
tp_sl_ratio = 1.75
risk_per_trade = 0.0046

In [2]:
def vwap(df):
    q = df.tick_volume
    p = (df.high + df.low + df.close) / 3
    date_groups = df.groupby(df.index.date)
    cum_vol = date_groups['tick_volume'].cumsum()
    cum_pv = (p * q).cumsum()
    vwap = cum_pv / cum_vol
    if len(vwap) > 1:
        vwap[1:] = vwap[:-1]
    else:
        vwap = vwap * 0
    df = df.assign(vwap=vwap)
    std = vwap.std()
    df = df.assign(upper_band=vwap +2*std, lower_band=vwap -2*std)
    return df

def isEngulfing(df, row):
    Length = len(df)
    High = list(df['high'])
    Low = list(df['low'])
    Close = list(df['close'])
    Open = list(df['open'])
    threshold = 0.0001
    if (row > 0 and Open[row-1] < Close[row-1] and Open[row] > Close[row] and 
        (Open[row] - Close[row-1]) >= -threshold and Close[row] < Open[row-1]):
        return 1 #bearish engulfing
    elif (row > 0 and Open[row-1] > Close[row-1] and Open[row] < Close[row] and 
        (Open[row] - Close[row-1]) <= +threshold and Close[row] > Open[row-1]):
        return 2 #bullish engulfing
    else:
        return 0

def generate_signal(df):
    last_row = len(df) - 1
    signal = isEngulfing(df, last_row)
    if signal == 1 and df.iloc[last_row]['close'] > df.iloc[last_row]['upper_band']:
        return 'sell'
    elif signal == 2 and df.iloc[last_row]['close'] < df.iloc[last_row]['lower_band']:
        return 'buy'
    else:
        return None

def get_vwap_and_engulfing_signal(df):
    df = vwap(df)
    signal = generate_signal(df)
    return signal

In [3]:
def close_position(position, deviation=20, magic=12345):
    order_type_dict = {
        0: mt5.ORDER_TYPE_SELL,
        1: mt5.ORDER_TYPE_BUY
    }

    price_dict = {
        0: mt5.symbol_info_tick(symbol).bid,
        1: mt5.symbol_info_tick(symbol).ask
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "position": position['ticket'],
        "symbol": symbol,
        "volume": volume,
        "type": order_type_dict[position['type']],
        "price": price_dict[position['type']],
        "deviation": deviation,
        "magic": magic,
        "comment": strategy_name,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    return order_result

def close_positions(order_type):
    order_type_dict = {
        'buy': 0,
        'sell': 1
    }

    positions = mt5.positions_get()

    if not positions:  # Check if there are open positions
        return

    positions_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())

    if order_type != 'all':
        positions_df = positions_df[(positions_df['type'] == order_type_dict[order_type])]

    for i, position in positions_df.iterrows():
        order_result = close_position(position)
        print('order_result: ', order_result)

def market_order(symbol, volume, order_type, stop_loss, take_profit, deviation=20, magic=12345):
    order_type_dict = {
        'buy': mt5.ORDER_TYPE_BUY,
        'sell': mt5.ORDER_TYPE_SELL
    }

    price_dict = {
        'buy': mt5.symbol_info_tick(symbol).ask,
        'sell': mt5.symbol_info_tick(symbol).bid
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type_dict[order_type],
        "price": price_dict[order_type],
        "sl": stop_loss,
        "tp": take_profit,
        "deviation": deviation,
        "magic": magic,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    return order_result

In [4]:
from datetime import datetime
import pytz

def check_allowed_trading_hours():
    # Define the timezone for the market you're trading in
    market_tz = pytz.timezone('Europe/Zurich')  # Switzerland timezone

    # Get the current time in the market's timezone
    now = datetime.now(market_tz)

    if 8 <= now.hour < 22:
        return True
    else:
        return False


In [6]:
if __name__ == '__main__':
    is_initialized = mt5.initialize()
    if not is_initialized:
        print('Failed to initialize MT5')
        exit(1)

    is_logged_in = mt5.login(login, password, server)
    if not is_logged_in:
        print('Failed to log in to MT5')
        exit(1)

    account_info = mt5.account_info()
    initial_account_balance = account_info.balance  # Save the initial account balance
    volume = round(initial_account_balance / 100000,4)  # Calculate volume


    while True:
        if not check_allowed_trading_hours():
            close_positions('all')
            print("Trading hours over. Shutting down.")
            break

        account_info = mt5.account_info()
        print(datetime.now(),
              '| Login: ', account_info.login,
              '| Balance: ', account_info.balance,
              '| Equity: ', account_info.equity)

        # Calculate the amount used for trading
        trading_balance = account_info.balance
        
        num_positions = mt5.positions_total()
        open_positions = mt5.positions_get(symbol=symbol)
        
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 30)
        df = pd.DataFrame(rates)
        
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.rename(columns={'time': 'timestamp'}, inplace=True)
        df.set_index('timestamp', inplace=True)
        
        signal = get_vwap_and_engulfing_signal(df)


        if signal == 'buy':
            if num_positions == 0 and check_allowed_trading_hours() and len(open_positions) == 0:
                entry_price = mt5.symbol_info_tick(symbol).ask
                stop_loss_pips = entry_price * risk_per_trade  # Risk per trade as a fraction of the entry price
                stop_loss = entry_price - stop_loss_pips
                take_profit = entry_price + (tp_sl_ratio * stop_loss_pips)
                lot_size = mt5.symbol_info(symbol).trade_contract_size
                order_result = market_order(symbol, volume, 'buy', stop_loss, take_profit)
                print(order_result)

        elif signal == 'sell':
            if num_positions == 0 and check_allowed_trading_hours() and len(open_positions) == 0:
                entry_price = mt5.symbol_info_tick(symbol).bid
                stop_loss_pips = entry_price * risk_per_trade  # Risk per trade as a fraction of the entry price
                stop_loss = entry_price + stop_loss_pips
                take_profit = entry_price - (tp_sl_ratio * stop_loss_pips)
                lot_size = mt5.symbol_info(symbol).trade_contract_size
                order_result = market_order(symbol, volume, 'sell', stop_loss, take_profit)
                print(order_result)

        time.sleep(60)


2024-11-03 17:48:35.287280 | Login:  90026240 | Balance:  10000.0 | Equity:  10000.0
2024-11-03 17:49:35.460801 | Login:  90026240 | Balance:  10000.0 | Equity:  10000.0
2024-11-03 17:50:35.570544 | Login:  90026240 | Balance:  10000.0 | Equity:  10000.0
2024-11-03 17:51:35.572196 | Login:  90026240 | Balance:  10000.0 | Equity:  10000.0
2024-11-03 17:52:35.587911 | Login:  90026240 | Balance:  10000.0 | Equity:  10000.0


KeyboardInterrupt: 